# Export slices containing metastases

This notebook goes through each step to export images of the metastases based on the segmentation

In [2]:
# import libraries

import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from pathlib import Path
from tqdm import tqdm
import pandas as pd

import imageio as iio
from PIL import Image

In [3]:
def save_slice_as_image(slice_data, output_path):
    # print(slice_data.shape)
    # print(np.mean(slice_data))
    # cv2.imwrite(output_path, slice_data)
    # plt.imsave(output_path, slice_data, cmap='gray')
    # plt.close()
    data = (slice_data * 255).astype(np.uint8)
    image = Image.fromarray(data)
    image.save(output_path)

# turn images 90 degress
def turn_image_90(image):
    return np.rot90(image, 3)

# mirror images
def mirror_image(image):
    return np.fliplr(image)

def process_nifty_mask(mask_path, mri_path, output_dir, patient_id=None, mri_sequence=None, save_mask_image = False, save_cut_out = False, area_threshold = 0, dataset = "regensburg"):

    # Load the NIFTI mask file
    img = nib.load(mask_path)
    mask_data = img.get_fdata()

    mri_data = nib.load(mri_path).get_fdata()

    unique_labels = np.unique(mask_data)
    #print(unique_labels)

    match dataset:
        case "regensburg":
            target_label = 1
        case "yale":

            if len(unique_labels) > 3:
                if 1 in unique_labels and 3 in unique_labels:
                    mask_data = np.where(mask_data == 3, 1, mask_data)
                target_label = 1 #3 #1
            else:
                target_label = 3
        case _:
            raise ValueError("Invalid dataset specified.")    

    new_unique_labels = np.unique(mask_data)
    #print(new_unique_labels)

    # Ensure the mask data is 3D
    if mask_data.ndim != 3:
        raise ValueError("The NIFTI mask data is not 3D.")

    # Find all connected components (metastases) with the target label in 3D
    labeled_mask = label(mask_data == target_label, connectivity=3)  # 3D connectivity
    regions = regionprops(labeled_mask)
    
    largest_slices = {}

    # Iterate through each region (metastasis)
    for region in regions:
        # Find the slice with the largest area within this region
        coords = region.coords
        slice_areas = {}
        for coord in coords:
            slice_idx = coord[2]  # Use the correct axis for slice indexing
            if slice_idx not in slice_areas:
                slice_areas[slice_idx] = 0
            slice_areas[slice_idx] += 1

        # Identify the slice index with the largest area for this metastasis
        largest_slice_idx = max(slice_areas, key=slice_areas.get)
        largest_area = slice_areas[largest_slice_idx]

        # Store information about the largest slice for each region
        slice_data = mask_data[:, :, largest_slice_idx]
        largest_slices[region.label] = (largest_area, slice_data, largest_slice_idx, region.label, coords)

    if len(largest_slices) == 0:
        print(f"No metastases found in {mri_path}")
        return
    
    # Save the largest slices as image files
    for label_idx, (area, slice_data, slice_idx, region_label, coords) in largest_slices.items():
        if area > area_threshold:
            full_slice_data = mri_data[:, :, slice_idx]
            mask_slice_data = np.zeros_like(slice_data)

            for coord in coords:
                if coord[2] == slice_idx:
                    mask_slice_data[coord[0], coord[1]] = target_label

            # Create the output path
            output_path = os.path.join(output_dir, f'{patient_id}_metastasis_{slice_idx}_{region_label}_{mri_sequence}.png')            
            save_slice_as_image(mirror_image(turn_image_90(full_slice_data)), output_path)

            if save_mask_image:
                mask_output_path = os.path.join(output_dir, f'{patient_id}_metastasis_{slice_idx}_{region_label}_{mri_sequence}_mask.png')

                # Create a new array with only the pixels of the target label
                mask_slice_data = np.zeros_like(mask_data)
                mask_slice_data[mask_data == target_label] = target_label
                mask_slice_data = mask_slice_data[:, :, slice_idx]

                save_slice_as_image(mirror_image(turn_image_90(mask_slice_data)), mask_output_path)
            

            if save_cut_out:

                for region in regions:
                    single_metastasis_mask = np.zeros_like(mask_data[:, :, slice_idx])
                    for coord in coords:
                        if coord[2] == slice_idx:
                            single_metastasis_mask[coord[0], coord[1]] = target_label
                    
                    masked_mri_slice = full_slice_data * (single_metastasis_mask == target_label)

                    # Create the output path
                    output_path = os.path.join(output_dir, f'{patient_id}_metastasis_{slice_idx}_{region_label}_{mri_sequence}_cutout.png')
                    save_slice_as_image(mirror_image(turn_image_90(masked_mri_slice)), output_path)

            #print(f"Saved largest slice for metastasis (slice {slice_idx}) with area {area} to {output_path}")

# Example usage
#mask_file_path = '/Users/LennartPhilipp/Desktop/testing_data/derivatives/segmented_AURORA_20240424-122816/sub-01009590/metastasis_unbinarized_floats.nii.gz'
#mri_path = "/Users/LennartPhilipp/Desktop/testing_data/derivatives/preprocessed_brainlesion_20240424-110551/sub-01009590/preprocessed/sub-01009590_fla_bet_normalized.nii.gz"
mask_file_path = "/Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/Pretreat-MetsToBrain-Masks/BraTS-MET-00100-000/BraTS-MET-00100-000-seg.nii.gz"
mri_path = "/Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_dataset_normalized/BraTS-MET-00100-000/n4_normalized_BraTS-MET-00100-000-t1c.nii.gz"
output_directory = '/Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices'
#os.makedirs(output_directory, exist_ok=True)
#process_nifty_mask(mask_file_path, mri_path, output_directory, save_mask_image=True, save_cut_out=True)

## create skript to go through all patients

### Regensburg Dataset

old code

In [17]:
# output: create folder for each patient and save the slices there

# input mri folder: path to all preprocessed patients
# go to each patient and get all 4 mri scans
# input segmentation folder: path to all segmented patients
# go to each patient and get metastatis_unbinarized_floats.nii.gz

path_to_output = Path("/Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices")
path_to_mri_patients = Path("/Users/LennartPhilipp/Desktop/testing_data/derivatives/preprocessed_brainlesion_20240424-110551")
path_to_segmentation_patients = Path("/Users/LennartPhilipp/Desktop/testing_data/derivatives/segmented_AURORA_20240424-122816")

segmentation_patients = [patient for patient in os.listdir(path_to_segmentation_patients) if os.path.isdir(os.path.join(path_to_segmentation_patients, patient))]

for patient in tqdm(segmentation_patients):
    # go through each patient
    
    print(f"currently working on {patient}")

    # create new folder for output for patient
    path_to_patient_output = Path(path_to_output) / Path(patient)
    os.makedirs(path_to_patient_output, exist_ok=True)

    # IMPORTANT!!!!!
    # adjust path for final run
    path_to_segmentation = Path(path_to_segmentation_patients) / Path(patient) / "metastasis_unbinarized_floats.nii.gz"

    path_to_mri_images = Path(path_to_mri_patients) / Path(patient) / "preprocessed"
    mri_images = [file for file in os.listdir(path_to_mri_images) if ".nii.gz" in file]

    for mri_image in mri_images:
        # go through each mri image for the patient

        mri_sequence = mri_image.split("_")[1]

        path_to_mri = Path(path_to_mri_images) / Path(mri_image)
        process_nifty_mask(path_to_segmentation, path_to_mri, path_to_patient_output, patient, mri_sequence)
    
    print(f"done with patient {patient}")
    

  0%|          | 0/5 [00:00<?, ?it/s]

currently working on sub-01009590
Saved largest slice for metastasis (slice 56) with area 424.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01009590/sub-01009590_metastasis_56_t2.png
Saved largest slice for metastasis (slice 56) with area 424.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01009590/sub-01009590_metastasis_56_fla.png
Saved largest slice for metastasis (slice 56) with area 424.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01009590/sub-01009590_metastasis_56_t1c.png


 20%|██        | 1/5 [00:00<00:01,  2.29it/s]

Saved largest slice for metastasis (slice 56) with area 424.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01009590/sub-01009590_metastasis_56_t1.png
done with patient sub-01009590
currently working on sub-01006290
Saved largest slice for metastasis (slice 63) with area 41.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01006290/sub-01006290_metastasis_63_t2.png
Saved largest slice for metastasis (slice 65) with area 33.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01006290/sub-01006290_metastasis_65_t2.png
Saved largest slice for metastasis (slice 64) with area 39.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01006290/sub-01006290_metastasis_64_t2.png
Saved largest slice for metastasis (slice 63) with area 41.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01006290/sub-01006290_metastasis_63_t1c.png
Saved largest slice for metastasis (slice 65) wi

 40%|████      | 2/5 [00:01<00:01,  1.71it/s]

...

Saved largest slice for metastasis (slice 120) with area 66.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005097/sub-01005097_metastasis_120_fla.png
Saved largest slice for metastasis (slice 120) with area 66.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005097/sub-01005097_metastasis_120_t2.png
done with patient sub-01005097
currently working on sub-01015961
Saved largest slice for metastasis (slice 60) with area 552.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_60_t2.png
Saved largest slice for metastasis (slice 43) with area 69.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_43_t2.png
Saved largest slice for metastasis (slice 60) with area 552.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_60_t1c.png
Saved largest slice for metastasis (slice 

 80%|████████  | 4/5 [00:02<00:00,  1.84it/s]

Saved largest slice for metastasis (slice 60) with area 552.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_60_fla.png
Saved largest slice for metastasis (slice 43) with area 69.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_43_fla.png
Saved largest slice for metastasis (slice 60) with area 552.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_60_t1.png
Saved largest slice for metastasis (slice 43) with area 69.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01015961/sub-01015961_metastasis_43_t1.png
done with patient sub-01015961
currently working on sub-01005630
Saved largest slice for metastasis (slice 41) with area 93.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_41_t2.png
Saved largest slice for metastasis (slice 33) 

100%|██████████| 5/5 [00:02<00:00,  1.86it/s]

Saved largest slice for metastasis (slice 41) with area 93.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_41_fla.png
Saved largest slice for metastasis (slice 33) with area 43.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_33_fla.png
Saved largest slice for metastasis (slice 41) with area 93.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_41_t1.png
Saved largest slice for metastasis (slice 33) with area 43.0 to /Users/LennartPhilipp/Desktop/testing_data/derivatives/met_slices/sub-01005630/sub-01005630_metastasis_33_t1.png
done with patient sub-01005630


working with actual segmented files

# IMPORTANT
some patients got processed multiple times, what was up with that?!

In [7]:
# output: create folder for each patient and save the slices there

# input mri folder: path to all preprocessed patients
# go to each patient and get all 4 mri scans
# input segmentation folder: path to all segmented patients
# go to each patient and get metastatis_unbinarized_floats.nii.gz

path_to_output = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Datensatz_RGB/regensburg_slices")
path_to_mri_patients = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Datensatz_RGB/preprocessed_n4_brainlesion_percentile_20240612-083743")
path_to_segmentation_patients = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Datensatz_RGB/Segs_Lennart_Export")

path_to_tsv = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/participants_01_08_24.tsv")

segmentation_patients = [patient for patient in os.listdir(path_to_segmentation_patients) if "labels" in patient]

pats_to_preprocess = pd.read_csv(path_to_tsv, sep="\t")["participant_id"].tolist()

already_processed_patients = [patient for patient in os.listdir(path_to_output) if os.path.isdir(os.path.join(path_to_output, patient))]

counter = 0

for patient in tqdm(segmentation_patients):
    # go through each patient

    # if counter >= 1:
    #     break
    
    # get patient ID
    patientID = patient.split("_")[0]

    # check if patient ID is in list of patients to preprocess
    if patientID not in pats_to_preprocess:
        continue

    if patientID in already_processed_patients:
        print(f"patient {patientID} already processed")
        continue

    print(f"currently working on {patientID}")

    # create new folder for output for patient
    path_to_patient_output = Path(path_to_output) / Path(patientID)
    os.makedirs(path_to_patient_output, exist_ok=True)

    # IMPORTANT!!!!!
    # adjust path for final run
    path_to_segmentation = Path(path_to_segmentation_patients) / Path(patient)

    path_to_mri_images = Path(path_to_mri_patients) / Path(patientID) / "perc_normalized"
    mri_images = [file for file in os.listdir(path_to_mri_images) if ".nii.gz" in file]

    if mri_images == []:
        print("********************************")
        print(f"patient {patientID} not complete")
        continue

    for mri_image in mri_images:
        # go through each mri image for the patient

        mri_sequence = mri_image.split("_")[1]

        path_to_mri = Path(path_to_mri_images) / Path(mri_image)
        process_nifty_mask(path_to_segmentation, path_to_mri, path_to_patient_output, patientID, mri_sequence,
                           save_mask_image = True,
                           save_cut_out = True,
                           area_threshold = 10)
    
    print(f"done with patient {patientID}")

    counter += 1

print(counter)
    

  0%|          | 0/484 [00:00<?, ?it/s]

currently working on sub-01134825


  0%|          | 1/484 [00:01<09:37,  1.20s/it]

done with patient sub-01134825
currently working on sub-01592849


  0%|          | 2/484 [00:02<09:10,  1.14s/it]

...

done with patient sub-01494236
currently working on sub-01104996


100%|█████████▉| 482/484 [12:09<00:02,  1.43s/it]

done with patient sub-01104996
currently working on sub-01565091


100%|██████████| 484/484 [12:10<00:00,  1.51s/it]

done with patient sub-01565091
471


In [23]:
path_to_image = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Datensatz_RGB/regensburg_test_slices/sub-01134825/sub-01134825_metastasis_105_1_t1.png")

test_image = iio.v3.imread(path_to_image)
print(test_image.shape)

(240, 240)


### Yale Dataset

In [66]:
# output: create folder for each patient and save the slices there

# input mri folder: path to all patient scans, including segmentations
# go to each patient and get all 4 mri scans as well as segmentation

path_to_output = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases")
path_to_yale_patients_raw = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/Pretreat-MetsToBrain-Masks")
path_to_yale_patients_processed = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_dataset_normalized")

yale_patients = [patient for patient in os.listdir(path_to_yale_patients_raw) if os.path.isdir(os.path.join(path_to_yale_patients_raw, patient))]

counter = 0

for patient in tqdm(yale_patients):

    if counter >= 5:
        break

    # go through each patient

    pat_id = patient.split("-")[2]
    
    print(f"currently working on {pat_id}")

    # create new folder for output for patient
    path_to_patient_output = Path(path_to_output) / Path(pat_id)
    os.makedirs(path_to_patient_output, exist_ok=True)

    # IMPORTANT!!!!!
    # adjust path for final run
    path_to_raw_patient_files = Path(path_to_yale_patients_raw) / Path(patient)
    segmentation_file = [file for file in os.listdir(path_to_raw_patient_files) if "seg" in file][0]
    path_to_segmentation = Path(path_to_raw_patient_files) / Path(segmentation_file)

    path_to_mri_images = Path(path_to_yale_patients_processed) / Path(patient)
    mri_images = [file for file in os.listdir(path_to_mri_images) if "normalized" in file]

    if len(mri_images) != 4:
        print("********************************")
        print(f"patient {patient} not complete")
        continue

    for mri_image in mri_images:
        # go through each mri image for the patient

        mri_sequence = mri_image.split("-")[4].split(".")[0]

        path_to_mri = Path(path_to_mri_images) / Path(mri_image)
        process_nifty_mask(
            mask_path = path_to_segmentation,
            mri_path = path_to_mri,
            output_dir = path_to_patient_output,
            patient_id = pat_id,
            mri_sequence = mri_sequence,
            save_mask_image = True,
            save_cut_out = True,
            dataset = "yale"
        )
        #process_nifty_mask(path_to_segmentation, path_to_mri, path_to_patient_output, patient_id = pat_id, mri_sequence)
    
    counter += 1

    print(f"done with patient {patient}")
    

  0%|          | 0/200 [00:00<?, ?it/s]

currently working on 00086
[0. 2. 3.]
[0. 2. 3.]
Saved largest slice for metastasis (slice 48) with area 33 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00086/00086_metastasis_48_1_t1c.nii.gz_cutout.png
Saved largest slice for metastasis (slice 50) with area 29 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00086/00086_metastasis_50_2_t1c.nii.gz_cutout.png
Saved largest slice for metastasis (slice 39) with area 43 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00086/00086_metastasis_39_3_t1c.nii.gz_cutout.png
[0. 2. 3.]
[0. 2. 3.]
Saved largest slice for metastasis (slice 48) with area 33 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00086/00086_metastasis_48_1_t2w.nii.gz_cutout.png
Saved largest slice for metastasis (slice 50) with area 29 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00086/00086_metastasis_50_2_t2w.nii.gz_cutout.png

  0%|          | 1/200 [00:02<08:29,  2.56s/it]

Saved largest slice for metastasis (slice 39) with area 43 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00086/00086_metastasis_39_3_t2f.nii.gz_cutout.png
done with patient BraTS-MET-00086-000
currently working on 00284
[0. 1. 2. 3.]
[0. 1. 2.]
Saved largest slice for metastasis (slice 60) with area 390 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00284/00284_metastasis_60_1_t2f.nii.gz_cutout.png
[0. 1. 2. 3.]
[0. 1. 2.]
Saved largest slice for metastasis (slice 60) with area 390 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00284/00284_metastasis_60_1_t1n.nii.gz_cutout.png
[0. 1. 2. 3.]
[0. 1. 2.]
Saved largest slice for metastasis (slice 60) with area 390 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00284/00284_metastasis_60_1_t2w.nii.gz_cutout.png
[0. 1. 2. 3.]
[0. 1. 2.]


  1%|          | 2/200 [00:03<06:13,  1.89s/it]

...

Saved largest slice for metastasis (slice 71) with area 715 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00290/00290_metastasis_71_1_t1c.nii.gz_cutout.png
done with patient BraTS-MET-00290-000
currently working on 00247
[0. 1. 2. 3.]
[0. 1. 2.]
Saved largest slice for metastasis (slice 31) with area 79 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00247/00247_metastasis_31_1_t1c.nii.gz_cutout.png
[0. 1. 2. 3.]
[0. 1. 2.]
Saved largest slice for metastasis (slice 31) with area 79 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00247/00247_metastasis_31_1_t2w.nii.gz_cutout.png
[0. 1. 2. 3.]
[0. 1. 2.]
Saved largest slice for metastasis (slice 31) with area 79 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00247/00247_metastasis_31_1_t1n.nii.gz_cutout.png
[0. 1. 2. 3.]
[0. 1. 2.]


  2%|▏         | 4/200 [00:06<05:03,  1.55s/it]

Saved largest slice for metastasis (slice 31) with area 79 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00247/00247_metastasis_31_1_t2f.nii.gz_cutout.png
done with patient BraTS-MET-00247-000
currently working on 00253
[0. 1. 2. 3.]
[0. 1. 2.]
Saved largest slice for metastasis (slice 63) with area 76 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00253/00253_metastasis_63_1_t1c.nii.gz_cutout.png
Saved largest slice for metastasis (slice 103) with area 117 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00253/00253_metastasis_103_2_t1c.nii.gz_cutout.png
Saved largest slice for metastasis (slice 73) with area 366 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00253/00253_metastasis_73_3_t1c.nii.gz_cutout.png
Saved largest slice for metastasis (slice 116) with area 53 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00253/00253_metastasis_11

  2%|▎         | 5/200 [00:10<06:33,  2.02s/it]

Saved largest slice for metastasis (slice 73) with area 366 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00253/00253_metastasis_73_3_t1n.nii.gz_cutout.png
Saved largest slice for metastasis (slice 116) with area 53 to /Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_metastases/00253/00253_metastasis_116_4_t1n.nii.gz_cutout.png
done with patient BraTS-MET-00253-000
